# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import math
import xlsxwriter
import requests

from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'L)EERNTLA(SGM CSALDAEKG/N  QAOSBT', 'calculationPrice': 'tops', 'open': 0, 'openTime': None, 'openSource': 'lfifoiac', 'close': 0, 'closeTime': None, 'closeSource': 'caffioli', 'high': 0, 'highTime': None, 'highSource': None, 'low': 0, 'lowTime': None, 'lowSource': None, 'latestPrice': 148.668, 'latestSource': 'IEX real time price', 'latestTime': '10:45:48 AM', 'latestUpdate': 1705001348588, 'latestVolume': None, 'iexRealtimePrice': 146.547, 'iexRealtimeSize': 100, 'iexLastUpdated': 1632870840138, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 152.29, 'previousVolume': 54421800, 'change': 0.077, 'changePercent': 0.00051, 'volume': None, 'iexMarketPercent': 0.0112460343762291, 'iexVolume': 190993, 'avgTotalVolume': 84602698, 'iexBidPrice': 1

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data["latestPrice"]
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()    
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )
        
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,157.390,53.12,N/A
1,AAL,20.716,-2.13,N/A
2,AAP,211.470,22.73,N/A
3,AAPL,153.030,29.12,N/A
4,ABBV,114.881,32.17,N/A
...,...,...,...,...
500,YUM,134.180,31.88,N/A
501,ZBH,155.100,34.27,N/A
502,ZBRA,578.671,39.97,N/A
503,ZION,56.900,5.6,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,315,MPC,59.720,4.91,N/A
1,503,ZION,56.900,5.6,N/A
2,63,BIO,767.660,5.65,N/A
3,381,PRU,105.380,5.84,N/A
4,203,GM,55.787,6.5,N/A
5,106,COF,165.100,6.59,N/A
6,95,CINF,121.820,6.83,N/A
7,17,AFL,58.190,6.84,N/A
8,87,CE,161.150,7.21,N/A
9,210,GS,416.100,7.43,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
    
final_dataframe

,index,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,315,MPC,59.720,4.91,334
1,503,ZION,56.900,5.6,351
2,63,BIO,767.660,5.65,26
3,381,PRU,105.380,5.84,189
4,203,GM,55.787,6.5,358
5,106,COF,165.100,6.59,121
6,95,CINF,121.820,6.83,164
7,17,AFL,58.190,6.84,343
8,87,CE,161.150,7.21,124
9,210,GS,416.100,7.43,48


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [13]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

#Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebita = data[symbol]['advanced-stats']['EBITDA']

ev_to_ebitda = enterprise_value/ebita

#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

29.71


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [21]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITA',
    'EV/EBITA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebita = data[symbol]['advanced-stats']['EBITDA']
        try:
            ev_to_ebitda = enterprise_value/ebita
        except TypeError:
            ev_to_ebitda = np.NaN
        
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                ],
                index = rv_columns
            ),
            ignore_index = True
        )
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITA,EV/EBITA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,156.300,N/A,54.1,N/A,10.08,N/A,8.19,N/A,33.075398,N/A,15.586922,N/A,N/A
1,AAL,20.970,N/A,-2.15,N/A,-1.84,N/A,0.7305,N/A,-4.763354,N/A,1.930531,N/A,N/A
2,AAP,211.760,N/A,23.01,N/A,4.0,N/A,1.26,N/A,11.274196,N/A,2.857008,N/A,N/A
3,AAPL,146.000,N/A,28.96,N/A,37.65,N/A,6.97,N/A,22.629984,N/A,17.147341,N/A,N/A
4,ABBV,119.460,N/A,31.4,N/A,16.27,N/A,3.88,N/A,11.073272,N/A,7.718268,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.842,N/A,31.39,N/A,-5.07,N/A,6.42,N/A,21.785397,N/A,10.750906,N/A,N/A
501,ZBH,150.519,N/A,34.4,N/A,2.49,N/A,4.06,N/A,20.176244,N/A,6.711561,N/A,N/A
502,ZBRA,572.000,N/A,39.5,N/A,12,N/A,6.04,N/A,30.097392,N/A,12.742594,N/A,N/A
503,ZION,55.644,N/A,5.46,N/A,1.2,N/A,2.65,N/A,4.436309,N/A,2.404411,N/A,N/A


{'week52change': 0.290414, 'week52high': 169.51, 'week52low': 90.41, 'marketcap': 75861250162, 'employees': 10724, 'day200MovingAvg': 134.17, 'day50MovingAvg': 148.64, 'float': 479955020, 'avg10Volume': 1641586.3, 'avg30Volume': 1479579.2, 'ttmEPS': 3.499, 'ttmDividendRate': 0.77, 'companyName': 'Zoetis, Inc.', 'sharesOutstanding': 494883276, 'maxChangePercent': 3.3525, 'year5ChangePercent': 1.69, 'year2ChangePercent': 0.608, 'year1ChangePercent': 0.281583, 'ytdChangePercent': 0.189876, 'month6ChangePercent': 0.10241, 'month3ChangePercent': 0.297048, 'month1ChangePercent': 0.159224, 'day30ChangePercent': 0.165634, 'day5ChangePercent': -0.012817, 'nextDividendDate': None, 'dividendYield': 0.00489706569361443, 'nextEarningsDate': '2020-10-13', 'exDividendDate': '2020-07-01', 'peRatio': 49.1, 'beta': 0.9584403132945124, 'totalCash': 1974321215, 'currentDebt': 549171552, 'revenue': 6518431250, 'grossProfit': 4133660573, 'totalRevenue': 6300683466, 'EBITDA': 2502772553, 'revenuePerShare': 1

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [22]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITA,EV/EBITA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,187.800,N/A,60.83,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,267.920,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,293.700,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.05,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,28.880,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.290,N/A,14.62,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.560,N/A,32.67,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,34.920,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2770.800,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.520,N/A,31.89,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [27]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [28]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITA,EV/EBITA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [30]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITA': 'EV/EBITA Percentile',
    'EV/GP': 'EV/GP Percentile'    
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
rv_dataframe        

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITA,EV/EBITA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,156.300,N/A,54.10,86.534653,10.08,79.405941,8.1900,81.782178,33.075398,88.316832,15.586922,83.762376,N/A
1,AAL,20.970,N/A,-2.15,9.108911,-1.84,7.524752,0.7305,6.732673,-4.763354,3.366337,1.930531,7.920792,N/A
2,AAP,211.760,N/A,23.01,42.970297,4.00,53.663366,1.2600,15.544554,11.274196,31.485149,2.857008,14.257426,N/A
3,AAPL,146.000,N/A,28.96,56.930693,37.65,96.039604,6.9700,77.227723,22.629984,72.871287,17.147341,86.930693,N/A
4,ABBV,119.460,N/A,31.40,61.584158,16.27,88.514851,3.8800,55.247525,11.073272,30.693069,7.718268,47.920792,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.842,N/A,31.39,61.386139,-5.07,4.356436,6.4200,74.455446,21.785397,70.891089,10.750906,68.712871,N/A
501,ZBH,150.519,N/A,34.40,67.524752,2.49,35.148515,4.0600,57.227723,20.176244,67.524752,6.711561,40.0,N/A
502,ZBRA,572.000,N/A,39.50,74.257426,12.00,82.970297,6.0400,69.306931,30.097392,85.544554,12.742594,76.039604,N/A
503,ZION,55.644,N/A,5.46,10.09901,1.20,13.168317,2.6500,38.811881,4.436309,5.346535,2.404411,11.683168,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [34]:
from statistics import mean
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'RV Score'] = mean([
        rv_dataframe.loc[row, 'PE Percentile'],
        rv_dataframe.loc[row, 'PB Percentile'],
        rv_dataframe.loc[row, 'PS Percentile'],
        rv_dataframe.loc[row, 'EV/EBITA Percentile'],
        rv_dataframe.loc[row, 'EV/GP Percentile']
    ])
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITA,EV/EBITA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,156.300,N/A,54.10,86.534653,10.08,79.405941,8.1900,81.782178,33.075398,88.316832,15.586922,83.762376,83.960396
1,AAL,20.970,N/A,-2.15,9.108911,-1.84,7.524752,0.7305,6.732673,-4.763354,3.366337,1.930531,7.920792,6.930693
2,AAP,211.760,N/A,23.01,42.970297,4.00,53.663366,1.2600,15.544554,11.274196,31.485149,2.857008,14.257426,31.584158
3,AAPL,146.000,N/A,28.96,56.930693,37.65,96.039604,6.9700,77.227723,22.629984,72.871287,17.147341,86.930693,78.0
4,ABBV,119.460,N/A,31.40,61.584158,16.27,88.514851,3.8800,55.247525,11.073272,30.693069,7.718268,47.920792,56.792079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.842,N/A,31.39,61.386139,-5.07,4.356436,6.4200,74.455446,21.785397,70.891089,10.750906,68.712871,55.960396
501,ZBH,150.519,N/A,34.40,67.524752,2.49,35.148515,4.0600,57.227723,20.176244,67.524752,6.711561,40.0,53.485149
502,ZBRA,572.000,N/A,39.50,74.257426,12.00,82.970297,6.0400,69.306931,30.097392,85.544554,12.742594,76.039604,77.623762
503,ZION,55.644,N/A,5.46,10.09901,1.20,13.168317,2.6500,38.811881,4.436309,5.346535,2.404411,11.683168,15.821782


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [37]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

/home/bhois/anaconda3/envs/algo-trading/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [40]:
portfolio_input()

Enter the value of your portfolio:2500000


In [46]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
    
rv_dataframe    

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITA,EV/EBITA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.038,1849,7.84,12.277228,0.4972,7.920792,0.4218,2.772277,3.506425,4.752475,0.413191,0.990099,5.742574
1,PRU,107.070,466,5.79,10.29703,0.6622,8.316832,0.6522,5.742574,4.712069,5.544554,0.655481,1.386139,6.257426
2,AIV,6.820,7331,-76.02,2.376238,1.9900,29.207921,-31.6400,0.19802,-44.589351,0.39604,-54.495944,0.792079,6.594059
3,AAL,20.970,2384,-2.15,9.108911,-1.8400,7.524752,0.7305,6.732673,-4.763354,3.366337,1.930531,7.920792,6.930693
4,AIG,53.368,936,-12.16,6.138614,0.7230,8.514851,1.0101,10.693069,6.291485,8.316832,0.984595,2.772277,7.287129
5,MCK,198.210,252,-7.11,7.524752,-58.6000,1.188119,0.1302,0.990099,8.161824,18.415842,2.866624,14.455446,8.514851
6,HFC,28.780,1737,-31.42,3.168317,0.9190,10.29703,0.4377,3.168317,7.463341,14.059406,2.510433,12.079208,8.554455
7,L,57.360,871,9.94,16.039604,0.8030,9.108911,0.9589,9.70297,5.469386,6.930693,0.937328,2.376238,8.831683
8,LNC,69.400,720,9.71,15.445545,0.5993,8.118812,0.6732,5.940594,7.442245,13.861386,0.661293,1.584158,8.990099
9,BA,241.150,207,-15.62,5.544554,-8.1300,3.762376,2.3000,33.960396,-31.376653,1.584158,-212.849772,0.39604,9.049505


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: